<a href="https://colab.research.google.com/github/yemre-data/DeepLearning_classification_skinlession_detection/blob/main/EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import cv2
from skimage import io
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler, normalize
import os
import glob
import missingno as msno
import random
from google.colab import files 
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from keras_preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from PIL import Image
import shutil
#importing other required libraries
import numpy as np
import pandas as pd
from sklearn.utils.multiclass import unique_labels
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
#from keras.utils import to_categorical
import torch
import torchvision
import torch.nn.functional as F
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
import pandas as pd
import numpy as np
import gc
import os
import cv2
import time
import datetime
import warnings
import random
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
def add_jpg(df):

  name_images = []
  for index, row in df.iterrows():
    name_images.append(row['name']+".jpg")
  df['name'] = name_images

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
train_dir = '/content/drive/My Drive/Skin_Disorders_Detection/train'
test_dir = '/content/drive/My Drive/Skin_Disorders_Detection/test'
train_df = pd.read_csv('/content/drive/My Drive/Skin_Disorders_Detection/train/train_df.csv')
df =  pd.read_csv('/content/drive/My Drive/Skin_Disorders_Detection/train/df.csv')
test_df = pd.read_csv('/content/drive/My Drive/Skin_Disorders_Detection/test/test_df.csv')
train_df.tail()


,name,meta.clinical.diagnosis
2980,combine_ISIC_0063778,squamous cell carcinoma
2981,combine_ISIC_0072652,squamous cell carcinoma
2982,combine_ISIC_0068354,squamous cell carcinoma
2983,combine_ISIC_0068843,squamous cell carcinoma
2984,combine_ISIC_0070260,squamous cell carcinoma


In [ ]:
add_jpg(train_df)
add_jpg(test_df)

In [ ]:
train_df['meta.clinical.diagnosis'] = train_df['meta.clinical.diagnosis'].astype('str')
test_df['meta.clinical.diagnosis'] = test_df['meta.clinical.diagnosis'].astype('str')


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
with tf.device('/device:GPU:0'):
    train_datagen = ImageDataGenerator(
      validation_split=0.20,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest'

    )

    train_generator = train_datagen.flow_from_dataframe(
      dataframe = train_df,
      directory = train_dir,
      x_col="name",
      y_col="meta.clinical.diagnosis",
      target_size=(380,380),
      subset="training",
      batch_size=16,
      class_mode="categorical"
    )

    val_generator = train_datagen.flow_from_dataframe(
      dataframe = train_df,
      directory = train_dir,
      x_col="name",
      y_col="meta.clinical.diagnosis",
      target_size=(380,380),
      subset="validation",
      batch_size=16,
      class_mode="categorical"
    )  

Found 2388 validated image filenames belonging to 3 classes.
Found 597 validated image filenames belonging to 3 classes.


In [ ]:
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

img_augmentation = Sequential(
    [
        preprocessing.RandomRotation(factor=0.15),
        preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
        preprocessing.RandomFlip(),
        preprocessing.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import EfficientNetB4

def build_model():
    inputs = layers.Input(shape=(380, 380, 3))
    x = img_augmentation(inputs)
    model = EfficientNetB4(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(3, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=4.0000e-05)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [ ]:
model= build_model()

71688192/71686520 [==============================] - 1s 0us/step


In [ ]:
from tensorflow.keras import layers

def unfreeze_model_and_add_layer(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    pre_model = model
    new_model = Sequential()
    new_model.add(pre_model)
    
    for layer in new_model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True
    new_model.add(layers.Dense(3, activation='softmax'))
    model = new_model
    optimizer = tf.keras.optimizers.Adam(learning_rate=4.0000e-05)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )

In [ ]:
unfreeze_model_and_add_layer(model)

In [ ]:
model.summary()

Model: "EfficientNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 380, 380, 3) 0                                            
__________________________________________________________________________________________________
img_augmentation (Sequential)   (None, 380, 380, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 380, 380, 3)  0           img_augmentation[0][0]           
__________________________________________________________________________________________________
normalization (Normalization)   (None, 380, 380, 3)  7           rescaling[0][0]                  
_______________________________________________________________________________________

In [ ]:

earlystopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

checkpointer = ModelCheckpoint(filepath="/content/drive/My Drive/Resized_Effcient_lr_00004_16.h5", verbose=1, save_best_only=True)

In [ ]:
start = datetime.datetime.now()
model_history = model.fit( train_generator, steps_per_epoch= train_generator.n //16 , epochs = 200, validation_data= val_generator, validation_steps= val_generator.n // 16, callbacks=[checkpointer, earlystopping])
end = datetime.datetime.now()
print("Time taken to run:", end-start)

Epoch 1/200
149/149 [==============================] - 1569s 10s/step - loss: 1.4040 - accuracy: 0.4073 - val_loss: 1.0343 - val_accuracy: 0.4662

Epoch 00001: val_loss improved from inf to 1.03428, saving model to /content/drive/My Drive/Resized_Effcient_lr_00004_16.h5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning:

Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.



Epoch 2/200
149/149 [==============================] - 99s 667ms/step - loss: 1.1407 - accuracy: 0.5143 - val_loss: 0.9915 - val_accuracy: 0.5270

Epoch 00002: val_loss improved from 1.03428 to 0.99154, saving model to /content/drive/My Drive/Resized_Effcient_lr_00004_16.h5
Epoch 3/200
149/149 [==============================] - 103s 696ms/step - loss: 1.0002 - accuracy: 0.5683 - val_loss: 0.9975 - val_accuracy: 0.5405

Epoch 00003: val_loss did not improve from 0.99154
Epoch 4/200
149/149 [==============================] - 103s 690ms/step - loss: 0.9322 - accuracy: 0.5915 - val_loss: 0.9770 - val_accuracy: 0.5557

Epoch 00004: val_loss improved from 0.99154 to 0.97699, saving model to /content/drive/My Drive/Resized_Effcient_lr_00004_16.h5
Epoch 5/200
149/149 [==============================] - 103s 693ms/step - loss: 0.8845 - accuracy: 0.6180 - val_loss: 0.9725 - val_accuracy: 0.5591

Epoch 00005: val_loss improved from 0.97699 to 0.97254, saving model to /content/drive/My Drive/Resize

In [ ]:
test_generator = ImageDataGenerator().flow_from_dataframe(dataframe = test_df[:304],directory = test_dir,target_size=(380,380),x_col="name",y_col="meta.clinical.diagnosis",class_mode='categorical',batch_size=16)

Found 304 validated image filenames belonging to 3 classes.


In [ ]:
test_df['meta.clinical.diagnosis'] = test_df['meta.clinical.diagnosis'].map( {'pigmented benign keratosis':2,'squamous cell carcinoma': 1, 'actinic keratosis': 0} ).astype(int)

In [ ]:
test_predict = model.predict(test_generator, steps = test_generator.n // 16, verbose =1)

19/19 [==============================] - 140s 8s/step


In [ ]:
predict = []

for i in test_predict:
  predict.append(str(np.argmax(i)))

predict = np.asarray(predict)

In [ ]:
original1 = np.asarray(test_df['meta.clinical.diagnosis'])[:len(predict)]
len(original1)

304

In [ ]:
original1 = original1.astype(float)

In [ ]:
original1 = original1.astype(int)

In [ ]:
original1 = np.round(original1, decimals = 4)

In [ ]:
original1 = original1.astype(str)

In [ ]:
accuracy = accuracy_score(original1, predict)
accuracy

0.29276315789473684

In [ ]:
def model_evaluation(model, history, train_images, train_labels,  test_images, test_labels, class_names=None, model_name=None):
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    epch = range(1, len(train_acc) + 1)
    plt.plot(epch, train_acc, 'g.', label='Training Accuracy')
    plt.plot(epch, val_acc, 'g', label='Validation acc')
    plt.title('Accuracy')
    plt.legend()
    plt.figure()
    plt.plot(epch, train_loss, 'r.', label='Training loss')
    plt.plot(epch, val_loss, 'r', label='Validation loss')
    plt.title('Loss')
    plt.legend()
    plt.show()
    
    results_test = model.evaluate(test_images, test_labels)
    print('Test Loss:', results_test[0])
    print('Test Accuracy:', results_test[1])
    
    y_train_pred = np.round(model.predict(train_images))
    y_pred = np.round(model.predict(test_images))
    print(classification_report(train_labels, y_train_pred,target_names=class_names))
    print(classification_report(test_labels, y_pred,target_names=class_names))
    confusion_matrix_plot(test_labels,y_pred)

In [ ]:
def confusion_matrix_plot(y_test,y_pred):
  f, axes = plt.subplots(3, 1, figsize=(25, 15))
  axes = axes.ravel()
  for i in range(3):

    disp = ConfusionMatrixDisplay(confusion_matrix(y_test[:, i],
                                                   y_pred[:, i]),
                                  display_labels=[0, i])
    disp.plot(ax=axes[i], values_format='.4g')
    disp.ax_.set_title(f'class {i}')
    if i<10:
        disp.ax_.set_xlabel('')
    if i%5!=0:
        disp.ax_.set_ylabel('')
    disp.im_.colorbar.remove()

  plt.subplots_adjust(wspace=0.10, hspace=0.1)
  f.colorbar(disp.im_, ax=axes)
  plt.show()

In [ ]:
model_evaluation(model, model.history,      class_names=['actinic keratosis','pigmented benign keratosis','squamous cell carcinoma'])